# Monte carlo simulation for NAEs hash collision

### Simulate the simultaneous collision using the IP, Port and IP&Port NAEs with a reduced number of NAEs and maximum hash value. The number of NAEs and maximum hash value has been scaled down with a factor of 100.

## Procedure
### Step 1
Create matrix (draw_matrix) with samples in rows and 12 columns, with the first 4 columns being the first draw (sample) for the source and destination addresses 
 and the second draw for source and destination adresses, the following 8 columns are similar columns for the Port and IP&Port NAEs
### Step 2
 Apply a hash function to draw_matrix using the maximum hash value (k). This produces the hashed_matrix.
### Step 3
 Detect a hit (collision) if the first and second draw from the source have the same hash value and at the same time 
 the first and second draw from the destination also have the same hash value. This procedure produce a matrix of three columns for the 
 hits on ip, port and ip_port addresses. This produces the hit_matrix.
### Step 4
 Calculate the probability of a simultaneous collision/hit in the ip, port and ip&port for the source and destiantion addresses.

In [2]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
%%R
library(digest)

hex_to_int = function(h) {
  # transform hex to int
  xx = strsplit(tolower(h), "")[[1L]]
  pos = match(xx, c(0L:9L, letters[1L:6L]))
  sum((pos - 1L) * 16^(rev(seq_along(xx) - 1)))
}


w=sample(1:100000, 1)
source_and_dest_draw <- function(num_max) {
  #draw a source and dest number from the range [1-num_max]
  w <<- w+1
  set.seed(w) #This is necessary because the clock time does not chage fast enough
  return(sample(1:num_max,2,replace=F))
}

hash_all <- function(vals) {
  # convert numbers to hash values 
  # the hashed valued are in the range [1-num_max]
  return(sapply(vals,hash_element) )
}


hash_element <- function(val) {
  # convert numbers to hash values 
  # the hashed valued are in the range [1-k]
  return(hex_to_int(digest(val, algo='xxhash32'))%% k) 
}


In [4]:
%%R
k = 30         # max hash value
num_ip = 90  # number of unique ip elements
num_port = 600  # number of unique portelements 
num_ip_port = 2000  # number of unique ip&pot elements

In [5]:
%%R

N_samples = 1e+7   # number of samples taken for the simulation
hit_matrix = matrix(NA,nrow=N_samples,ncol=3)
colnames(hit_matrix) <- c("ip_hit", "port_hit", "ip_port_hot")
hashed_matrix = matrix(NA,nrow=N_samples,ncol=12)
colnames(hashed_matrix) <- c("ip_s_1", "ip_d_1", "ip_s_2","ip_d_2","port_s_1", "port_d_1", "port_s_2","port_d_2", "ip_port_s_1", "ip_port_d_1", "ip_port_s_2","ip_port_d_2")
draw_matrix = matrix(NA,nrow=N_samples,ncol=12)
colnames(draw_matrix) <- c("ip_s_1", "ip_d_1", "ip_s_2","ip_d_2","port_s_1", "port_d_1", "port_s_2","port_d_2", "ip_port_s_1", "ip_port_d_1", "ip_port_s_2","ip_port_d_2")
for (i in 1:N_samples){
  ip_draw = c( source_and_dest_draw(num_ip), source_and_dest_draw(num_ip) ) # Step 1. For IP addresses
  ip_hashed = hash_all(ip_draw)                                             # Step 2. For IP addresses
  ip_hit = (ip_hashed[1]==ip_hashed[3]) & (ip_hashed[2]==ip_hashed[4])      # Step 3. For IP addresses

  port_draw = c( source_and_dest_draw(num_port), source_and_dest_draw(num_port) ) # Step 1. For Port addresses
  port_hashed = hash_all(port_draw)                                               # Step 2. For Port addresses
  port_hit = (port_hashed[1]==port_hashed[3]) & (port_hashed[2]==port_hashed[4])  # Step 3. For Port addresses
  
  ip_port_draw = c( source_and_dest_draw(num_ip_port), source_and_dest_draw(num_ip_port) )       # Step 1. For IP_Port addresses
  ip_port_hashed = hash_all(ip_port_draw)                                                        # Step 2. For IP_Port addresses
  ip_port_hit = (ip_port_hashed[1]==ip_port_hashed[3]) & (ip_port_hashed[2]==ip_port_hashed[4])  # Step 3. For IP_Port addresses

  #build the final matrices
  hit_matrix[i,] = cbind(ip_hit,port_hit,ip_port_hit)
  hashed_matrix[i,] = cbind(ip_hashed,port_hashed,ip_port_hashed)
  draw_matrix[i,] = cbind(ip_draw,port_draw,ip_port_draw)
}


vector_total_hits = apply(hit_matrix, 1, all)   # Step 4. Calculate the probability of a simultaneous hit for all NAEs
prob_total_hit = mean(vector_total_hits)        # Step 4
print(paste0("Probability total hit : ",prob_total_hit))



[1] "Probability total hit : 0"


In [ ]:
%%R

# Probabilities of collision on IP, Port and IP&Port separately
prob_ip_hit = mean(hit_matrix[,1]) 
print(paste0("Probability IP hit : ",prob_ip_hit))
prob_port_hit = mean(hit_matrix[,2]) 
print(paste0("Probability Port hit : ",prob_port_hit))
prob_ip_port_hit = mean(hit_matrix[,3]) 
print(paste0("Probability IP&Port hit : ",prob_ip_port_hit))

[1] "Probability IP hit : 0.0022053"
[1] "Probability Port hit : 0.0011835"
[1] "Probability IP&Port hit : 0.0011342"


In [ ]:
%%R
# Theoretical probability for a single NAE
print((1/k)**2)
# Theoretical probability for all 3 NAEs
print((1/k)**6)

[1] 0.001111111
[1] 1.371742e-09


In [ ]:
%%R
hit_matrix[1:20,]

      ip_hit port_hit ip_port_hot
 [1,]  FALSE    FALSE       FALSE
 [2,]  FALSE    FALSE       FALSE
 [3,]  FALSE    FALSE       FALSE
 [4,]  FALSE    FALSE       FALSE
 [5,]  FALSE    FALSE       FALSE
 [6,]  FALSE    FALSE       FALSE
 [7,]  FALSE    FALSE       FALSE
 [8,]  FALSE    FALSE       FALSE
 [9,]  FALSE    FALSE       FALSE
[10,]  FALSE    FALSE       FALSE
[11,]  FALSE    FALSE       FALSE
[12,]  FALSE    FALSE       FALSE
[13,]  FALSE    FALSE       FALSE
[14,]  FALSE    FALSE       FALSE
[15,]  FALSE    FALSE       FALSE
[16,]  FALSE    FALSE       FALSE
[17,]  FALSE    FALSE       FALSE
[18,]  FALSE    FALSE       FALSE
[19,]  FALSE    FALSE       FALSE
[20,]  FALSE    FALSE       FALSE


In [ ]:
%%R
draw_matrix[1:20,]

      ip_s_1 ip_d_1 ip_s_2 ip_d_2 port_s_1 port_d_1 port_s_2 port_d_2
 [1,]     64      3     49     20      216      384      264      338
 [2,]     53     22     46     64      137      562      245       99
 [3,]     45     49     19     54      488      466      328       77
 [4,]     68     42     73      9      256      580      517      418
 [5,]     50     51      6     61      149      205      560      231
 [6,]     17      3      4      7      354      283       24      130
 [7,]     30     13     15     18      425      552      439       81
 [8,]     69     50      5     56      228        8      550      333
 [9,]     36     65     23     37      120       25      249      374
[10,]     83     87     18     70      586       71      161      130
[11,]     17     72     58     78      442      486      438      399
[12,]     38     87     56     42      581      506      393       44
[13,]     60     74     79      5      576      577        1        5
[14,]     71     45 

In [ ]:
%%R
hashed_matrix[1:20,]

      ip_s_1 ip_d_1 ip_s_2 ip_d_2 port_s_1 port_d_1 port_s_2 port_d_2
 [1,]      4     27     21     22       29       10        7        8
 [2,]     10      5      5      4        3       24       15       18
 [3,]      6     21     23     17       24       19        5       13
 [4,]     12     28     17     20       14       23       29       10
 [5,]     24     22     17     22       11       11        1       17
 [6,]     16     27      6     17       20        0        9       13
 [7,]     28      9     26     13        4       16       28       11
 [8,]     27     24      0      2       24       27       16       27
 [9,]     22      6      4     17       22        6       20       18
[10,]     17      2     13      6        3       27       16       13
[11,]     16     29     11     20       25       23       18       28
[12,]      0      2      2     28       15       14       26        0
[13,]      7      2      2      0       29       19        3        0
[14,]     27      6 